In [1]:
# Data manipulation imports
import pandas as pd
import numpy as np

# Graphing imports
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in data from datasets folder
df_demographics = pd.read_csv('datasets/demographics_by_country.csv')
df_education = pd.read_csv('datasets/education_by_country.csv')
df_gdp = pd.read_csv('datasets/gdp_by_country.csv')
df_happiness = pd.read_csv('datasets/happiness_by_country.csv')
df_life_expectancy = pd.read_csv('datasets/life_expectancy_by_country.csv')
df_inequality = pd.read_csv('datasets/inequality_by_country_gini_data.csv')
df_income = pd.read_csv('datasets/median_income_by_country_data.csv')

In [3]:
df_demographics.head()

,cca2,name,pop2021,pop2020,pop2050,pop2030,pop2019,pop2015,pop2010,pop2000,pop1990,pop1980,pop1970,area,Density,GrowthRate,WorldPercentage,rank
0,CN,China,1444216.107,1439323.776,1402405.170,1464340.159,1433783.686,1406847.870,1368810.615,1290550.765,1176883.674,1000089.235,827601.394,9706961,148.7815,1.0034,0.1834,1
1,IN,India,1393409.038,1380004.385,1639176.033,1503642.322,1366417.754,1310152.403,1234281.170,1056575.549,873277.798,698952.844,555189.792,3287590,423.8391,1.0097,0.1769,2
2,US,United States,332915.073,331002.651,379419.102,349641.876,329064.917,320878.310,309011.475,281710.909,252120.309,229476.354,209513.341,9372610,35.5200,1.0058,0.0423,3
3,ID,Indonesia,276361.783,273523.615,330904.664,299198.430,270625.568,258383.256,241834.215,211513.823,181413.402,147447.836,114793.178,1904569,145.1046,1.0104,0.0351,4
4,PK,Pakistan,225199.937,220892.340,338013.196,262958.794,216565.318,199426.964,179424.641,142343.578,107647.921,78054.343,58142.060,881912,255.3542,1.0195,0.0286,5


In [4]:
df_demographics = df_demographics.rename(columns = {'name' : 'country',
                                                    'pop2021' :'population',
                                                    'area' : 'area_square_km',
                                                    'Density' : 'density_square_km'})

df_demographics = df_demographics[['country', 'population', 'area_square_km','density_square_km']]
df_demographics.head()

,country,population,area_square_km,density_square_km
0,China,1444216.107,9706961,148.7815
1,India,1393409.038,3287590,423.8391
2,United States,332915.073,9372610,35.5200
3,Indonesia,276361.783,1904569,145.1046
4,Pakistan,225199.937,881912,255.3542


In [5]:
df_education.head()

,Entity,Code,Year,Political Regime (OWID based on Polity IV and Wimmer & Min),"Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))",Year.1,"Total population (Gapminder, HYDE & UN)",Continent
0,Abkhazia,OWID_ABK,2015.0,NaN,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1816.0,-6.0,0.35,1970.0,3296506.0,NaN
2,Afghanistan,AFG,1817.0,-6.0,0.35,1970.0,3302044.0,NaN
3,Afghanistan,AFG,1818.0,-6.0,0.35,1970.0,3308390.0,NaN
4,Afghanistan,AFG,1819.0,-6.0,0.35,1970.0,3315547.0,NaN


In [6]:
df_education = df_education[df_education['Year'] == 2015]

df_education = df_education.rename(columns = {'Entity' : 'country',
                                              'Political Regime (OWID based on Polity IV and Wimmer & Min)' :'auto_demo_polity',
                                              'Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))' : 'average_years_of_school'})

df_education = df_education[['country', 'average_years_of_school', 'auto_demo_polity']]
df_education.head()

,country,average_years_of_school,auto_demo_polity
0,Abkhazia,NaN,NaN
177,Afghanistan,0.35,-1.0
338,Africa,NaN,NaN
343,Akrotiri and Dhekelia,NaN,NaN
542,Albania,3.31,9.0


In [7]:
df_gdp.head()

,Entity,Code,Year,Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),Political Regime (OWID based on Polity IV and Wimmer & Min),"Total population (Gapminder, HYDE & UN)",Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1816,NaN,-6.0,3296506.0,NaN
2,Afghanistan,AFG,1817,NaN,-6.0,3302044.0,NaN
3,Afghanistan,AFG,1818,NaN,-6.0,3308390.0,NaN
4,Afghanistan,AFG,1819,NaN,-6.0,3315547.0,NaN


In [8]:
df_gdp = df_gdp[df_gdp['Year'] == 2015]

df_gdp = df_gdp.rename(columns = {'Entity' : 'country',
                                  'Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019))' :  'gdp_per_capita'})

df_gdp = df_gdp[['country', 'gdp_per_capita']]
df_gdp.head()

,country,gdp_per_capita
0,Abkhazia,NaN
177,Afghanistan,NaN
338,Africa,NaN
343,Akrotiri and Dhekelia,NaN
389,Albania,11228.951


Only keep the 'Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019))' column for 2015 data, transfer to snake-case

In [9]:
# Some of the data is already standard-scaled
df_happiness.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


Only keep the 'Happiness_Score' column, transfer to snake-case

In [10]:
df_happiness = df_happiness.rename(columns = {'Country' : 'country',
                                              'Happiness Score' : 'happiness_index',
                                              'Freedom' : 'freedom_index',
                                              'Trust (Government Corruption)' : 'less_corruption_index'})

df_happiness = df_happiness[['country', 'happiness_index', 'freedom_index', 'less_corruption_index']]
df_happiness.head()

,country,happiness_index,freedom_index,less_corruption_index
0,Switzerland,7.587,0.66557,0.41978
1,Iceland,7.561,0.62877,0.14145
2,Denmark,7.527,0.64938,0.48357
3,Norway,7.522,0.66973,0.36503
4,Canada,7.427,0.63297,0.32957


In [11]:
df_life_expectancy.head()

,Entity,Code,Year,Life expectancy
0,Afghanistan,AFG,1950,27.638
1,Afghanistan,AFG,1951,27.878
2,Afghanistan,AFG,1952,28.361
3,Afghanistan,AFG,1953,28.852
4,Afghanistan,AFG,1954,29.350


Only keep the 'Life expectancy' column for 2015 data, transfer to snake-case

In [12]:
df_life_expectancy = df_life_expectancy[df_life_expectancy['Year'] == 2015]

df_life_expectancy = df_life_expectancy.rename(columns = {'Entity' : 'country',
                                                          'Life expectancy' :  'life_expectancy_years'})

df_life_expectancy = df_life_expectancy[['country', 'life_expectancy_years']]
df_life_expectancy.head()

,country,life_expectancy_years
65,Afghanistan,63.377
137,Africa,61.607
207,Albania,78.025
280,Algeria,76.090
350,American Samoa,73.588


In [13]:
df_inequality.head()

,name,slug,value,date_of_information,ranking,region
0,South Africa,south-africa,63.0,2014 est.,1,Africa
1,Namibia,namibia,59.1,2015 est.,2,Africa
2,Zambia,zambia,57.1,2015 est.,3,Africa
3,Sao Tome and Principe,sao-tome-and-principe,56.3,2017 est.,4,Africa
4,Eswatini,eswatini,54.6,2016 est.,5,Africa


In [14]:
df_inequality = df_inequality.rename(columns = {'name' : 'country',
                                                'value' :  'inequality_gini'})

df_inequality = df_inequality[['country', 'inequality_gini']]
df_inequality.head()

,country,inequality_gini
0,South Africa,63.0
1,Namibia,59.1
2,Zambia,57.1
3,Sao Tome and Principe,56.3
4,Eswatini,54.6


In [15]:
df_income.head()

,country,medianHouseholdIncome,medianPerCapitaIncome,medianAnnualIncome,pop2021
0,Luxembourg,52493.0,18418.0,73910.0,634.814
1,Norway,51489.0,19308.0,82500.0,5465.630
2,Sweden,50514.0,18632.0,55840.0,10160.169
3,Australia,46555.0,15026.0,54910.0,25788.215
4,Denmark,44360.0,18262.0,63240.0,5813.298


In [16]:
df_income = df_income.rename(columns = {'medianHouseholdIncome' : 'median_household_income_dollars',
                                        'medianPerCapitaIncome' :  'median_per_capita_income_dollars',
                                        'medianAnnualIncome' : 'median_individual_income_dollars'})

df_income = df_income[['country', 'median_household_income_dollars', 'median_per_capita_income_dollars', 'median_individual_income_dollars']]
df_income.head()

,country,median_household_income_dollars,median_per_capita_income_dollars,median_individual_income_dollars
0,Luxembourg,52493.0,18418.0,73910.0
1,Norway,51489.0,19308.0,82500.0
2,Sweden,50514.0,18632.0,55840.0
3,Australia,46555.0,15026.0,54910.0
4,Denmark,44360.0,18262.0,63240.0


## Merge Dataframes
Merge all of the different datasets into one

In [24]:
gov = pd.merge(left = df_demographics, right = df_education, on = 'country')
gov = pd.merge(left = gov, right = df_gdp, on = 'country' )
gov = pd.merge(left = gov, right = df_happiness, on = 'country')
gov = pd.merge(left = gov, right = df_life_expectancy, on = 'country')
gov = pd.merge(left = gov, right = df_inequality, on = 'country' )
gov = pd.merge(left = gov, right = df_income, on = 'country')
gov.head(30)

,country,population,area_square_km,density_square_km,average_years_of_school,auto_demo_polity,gdp_per_capita,happiness_index,freedom_index,less_corruption_index,life_expectancy_years,inequality_gini,median_household_income_dollars,median_per_capita_income_dollars,median_individual_income_dollars
0,China,1444216.107,9706961,148.7815,2.77,-7.0,12009.4430,5.140,0.51697,0.02781,75.928,38.5,6180.0,1786.0,10410.0
1,India,1393409.038,3287590,423.8391,1.24,9.0,5722.5186,4.565,0.39786,0.08492,68.607,35.7,3168.0,616.0,2130.0
2,United States,332915.073,9372610,35.5200,10.61,10.0,53533.2810,7.119,0.54604,0.15890,78.910,41.1,43585.0,15480.0,65760.0
3,Indonesia,276361.783,1904569,145.1046,2.26,9.0,9781.2070,5.399,0.46611,0.00000,70.768,37.8,2199.0,541.0,4050.0
4,Pakistan,225199.937,881912,255.3542,1.31,7.0,4815.9243,5.194,0.12102,0.10464,66.577,33.5,4060.0,480.0,1530.0
5,Brazil,213993.437,8515767,25.1291,3.09,8.0,14272.8540,6.983,0.49049,0.17521,74.994,53.9,7522.0,2247.0,9130.0
6,Nigeria,211400.708,923768,228.8461,NaN,7.0,4965.5610,5.268,0.34334,0.04030,53.112,35.1,2667.0,493.0,2030.0
7,Bangladesh,166303.498,147570,1126.9465,1.16,1.0,3114.0554,4.694,0.40820,0.12569,71.514,32.4,2819.0,567.0,1940.0
8,Russia,145912.025,17098242,8.5337,5.26,4.0,22832.8630,5.716,0.36679,0.03005,71.464,37.5,11724.0,4129.0,11260.0
9,Mexico,130262.216,1964375,66.3123,2.89,8.0,16903.8200,7.187,0.48181,0.21312,74.904,36.8,11680.0,2900.0,9430.0


## Data overview
Check for overviews of the data before imputing

In [18]:
gov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 125
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   country                           126 non-null    object 
 1   population                        126 non-null    float64
 2   area_square_km                    126 non-null    int64  
 3   density_square_km                 126 non-null    float64
 4   average_years_of_school           109 non-null    float64
 5   auto_demo_polity                  120 non-null    float64
 6   gdp_per_capita                    125 non-null    float64
 7   happiness_index                   126 non-null    float64
 8   freedom_index                     126 non-null    float64
 9   less_corruption_index             126 non-null    float64
 10  life_expectancy_years             126 non-null    float64
 11  inequality_gini                   126 non-null    float64
 12  median_h

In [19]:
gov.isnull().sum() #missing values for gdp, school, and auto_demo

country                              0
population                           0
area_square_km                       0
density_square_km                    0
average_years_of_school             17
auto_demo_polity                     6
gdp_per_capita                       1
happiness_index                      0
freedom_index                        0
less_corruption_index                0
life_expectancy_years                0
inequality_gini                      0
median_household_income_dollars      4
median_per_capita_income_dollars     4
median_individual_income_dollars    56
dtype: int64

In [20]:
gov.isnull().sum().sum()

88

In [21]:
gov.describe()

,population,area_square_km,density_square_km,average_years_of_school,auto_demo_polity,gdp_per_capita,happiness_index,freedom_index,less_corruption_index,life_expectancy_years,inequality_gini,median_household_income_dollars,median_per_capita_income_dollars,median_individual_income_dollars
count,1.260000e+02,1.260000e+02,126.000000,109.000000,120.000000,125.000000,126.000000,126.000000,126.000000,126.000000,126.000000,122.000000,122.000000,70.000000
mean,5.831055e+04,9.549778e+05,260.975326,3.966697,5.233333,19115.074617,5.450905,0.430080,0.142531,73.123786,37.319841,12599.934426,3896.540984,24107.142857
std,1.821544e+05,2.314626e+06,957.051078,2.724557,5.623679,18464.330398,1.177540,0.157006,0.124552,7.594370,7.328203,12865.220644,4820.751979,24240.146849
min,3.433530e+02,3.160000e+02,2.128600,0.010000,-10.000000,781.909550,2.839000,0.000000,0.000000,52.941000,24.200000,571.000000,47.000000,540.000000
25%,6.545963e+03,6.537750e+04,33.742425,1.450000,2.750000,4965.561000,4.566500,0.328330,0.059275,68.614500,32.700000,3551.250000,616.250000,4347.500000
50%,1.509575e+04,2.384620e+05,90.919250,3.380000,7.500000,13594.711000,5.277000,0.434635,0.104525,74.913500,35.950000,7437.500000,1790.500000,11480.000000
75%,4.289495e+04,7.766970e+05,149.173675,5.790000,10.000000,25939.902000,6.444000,0.556168,0.175060,79.401250,41.325000,16219.750000,5095.500000,43425.000000
max,1.444216e+06,1.709824e+07,8305.191500,10.690000,10.000000,98941.203000,7.587000,0.669730,0.551910,84.043000,63.000000,52493.000000,19308.000000,85500.000000


## Impute Missing Data and Export
I will just impute with the median given how there are only 88 missing values in the whole dataset, if there were more I might experiment with more advanced imputation methods 

In [22]:
gov['average_years_of_school'] = gov['average_years_of_school'].fillna(gov['average_years_of_school'].dropna().median())
gov['auto_demo_polity'] = gov['auto_demo_polity'].fillna(gov['auto_demo_polity'].dropna().median())
gov['gdp_per_capita'] = gov['gdp_per_capita'].fillna(gov['gdp_per_capita'].dropna().median())
gov['median_household_income_dollars'] = gov['median_household_income_dollars'].fillna(gov['median_household_income_dollars'].dropna().median())
gov['median_per_capita_income_dollars'] = gov['median_per_capita_income_dollars'].fillna(gov['median_per_capita_income_dollars'].dropna().median())
gov['median_individual_income_dollars'] = gov['median_individual_income_dollars'].fillna(gov['median_individual_income_dollars'].dropna().median())

gov.isnull().sum().sum()

0

In [23]:
gov.to_csv('gov.csv', index = False)